In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import statsmodels.api as sm
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.evaluation import HierarchicalEvaluation
from hierarchicalforecast.methods import BottomUp, TopDown, MinTrace
from sklearn.metrics import mean_squared_error, mean_absolute_error
import holidays
from prophet import Prophet
import pickle

sns.set_theme(style="darkgrid")

load = pd.read_csv('data/load.csv')

c:\Users\louis\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data preprocessing

In [3]:
load_df = pd.read_csv('data/load.csv')
hierarchy_df = pd.read_csv("data/hierarchy.csv")
humidity_df = pd.read_csv("data/relative humidity.csv")
temperature_df = pd.read_csv("data/temperature.csv")

In [4]:
def df_formatting(load_df, hierarchy_df, humidity_df, temperature_df):
    load_df = pd.melt(load_df, id_vars=["meter_id", "date"], value_vars=load_df.columns.difference(["meter_id", "date"]),
                                var_name="hour", value_name="load")
    load_df["hour"] = load_df["hour"].str.strip("h").astype(int) - 1
    load_df["timestamp"] = pd.to_datetime(load_df["date"] + " " + load_df["hour"].astype(str) + ":00:00", format="%m/%d/%Y %H:%M:%S")
    load_df["meter_id"] = load_df["meter_id"].astype(int)
    load_df = load_df.drop(columns=["date", "hour"])
    # Remove meter ids that appear in training but not in test and conversely
    aggregate_list = [('max_timestamp', 'max'), ('min_timestamp', 'min')]
    meters_df = load_df.groupby("meter_id")["timestamp"].agg(aggregate_list).reset_index()
    excluded_meters_df = meters_df[(meters_df["max_timestamp"]<dt.datetime(2011,1,1)) | (meters_df["min_timestamp"]>dt.datetime(2011,1,1))]
    excluded_meters = excluded_meters_df["meter_id"].to_list() + [236]
    load_df = load_df[~load_df["meter_id"].isin(excluded_meters)]
    # Ensure that all training points are present, being possibly 0
    date_rng = pd.date_range(start='2005-01-01', end='2010-12-31 23:00:00', freq='1H')
    all_combinations = pd.MultiIndex.from_product([load_df['meter_id'].unique(), date_rng], names=['meter_id', 'timestamp'])
    df_all_combinations = pd.DataFrame(index=all_combinations).reset_index()
    df_filled = pd.merge(df_all_combinations, load_df, on=['meter_id', 'timestamp'], how='left', suffixes=('_orig', '')).fillna(0)
    df_filled = pd.concat([df_filled, load_df[load_df["timestamp"]>=dt.datetime(2011,1,1)]])
    df_filled = df_filled.sort_values(by=["meter_id", "timestamp"]).reset_index(drop=True)
    data_df = df_filled.merge(hierarchy_df, on="meter_id", how="left")
    data_df[hierarchy_df.columns.difference(["meter_id"])] = data_df[hierarchy_df.columns.difference(["meter_id"])].astype(str)

    humidity_df["timestamp"] = pd.to_datetime(humidity_df["date"] + " " + (humidity_df["hr"] - 1).astype(str) + ":00:00", format="%d%b%Y %H:%M:%S")
    temperature_df["timestamp"] = pd.to_datetime(temperature_df["date"] + " " + (temperature_df["hr"] - 1).astype(str) + ":00:00", format="%d%b%Y %H:%M:%S")
    humidity_df = humidity_df.drop(columns=["date", "hr"])
    temperature_df = temperature_df.drop(columns=["date", "hr"])
    temperature_df["avg_temperature"] = temperature_df.filter(like="t_").mean(axis=1)
    humidity_df["avg_humidity"] = humidity_df.filter(like="rh_").mean(axis=1)
    temperature_df = temperature_df[["timestamp", "avg_temperature"]]
    humidity_df = humidity_df[["timestamp", "avg_humidity"]]
    data_df = data_df.merge(humidity_df, on="timestamp", how="left")
    data_df = data_df.merge(temperature_df, on="timestamp", how="left")
    return data_df

joined_data_df = df_formatting(load_df, hierarchy_df, humidity_df, temperature_df)

In [5]:
def get_datasets(df):
  aggregation_dict = {"load": "sum"}
  aggregation_dict.update({col: "first" for col in df.columns if col not in ["load", "timestamp"]})
  top_level_df = df.groupby("timestamp").agg(aggregation_dict).reset_index()
  aggregate_level_names = pd.unique(df["aggregate"])
  aggregate_levels = {}
  for agg_level in aggregate_level_names:
    agg_level_df = df[df["aggregate"] == agg_level].copy()
    agg_level_df = agg_level_df.groupby("timestamp").agg(aggregation_dict).reset_index()
    aggregate_levels[agg_level] = agg_level_df
  mid_level_names = pd.unique(df["mid_level"])
  mid_levels = {}
  for mid_level in mid_level_names:
    mid_level_df = df[df["mid_level"] == mid_level].copy()
    mid_level_df = mid_level_df.copy().groupby("timestamp").agg(aggregation_dict).reset_index()
    mid_levels[mid_level] = mid_level_df
  bottom_level_names = pd.unique(df["meter_id"])
  bottom_levels = {}
  for bottom_level in bottom_level_names:
    bottom_level_df = df[df["meter_id"] == bottom_level].copy()
    bottom_level_df = bottom_level_df.copy().groupby("timestamp").agg(aggregation_dict).reset_index()
    bottom_levels[bottom_level] = bottom_level_df
  return top_level_df, aggregate_levels, mid_levels, bottom_levels

In [6]:
top_level_df, aggregate_levels, mid_levels, bottom_levels = get_datasets(joined_data_df)

In [7]:
def feature_engineering(df, encode_meter_id):
    # Basic feature engineering: indicator of day of the week, month, is_holiday (in MA), one-hot encoding of hierarchies and of the meter id
    df['year'] = df['timestamp'].dt.year
    df['cos_month'] = np.cos(2*np.pi*(df['timestamp'].dt.month/12))
    df['sin_month'] = np.sin(2*np.pi*(df['timestamp'].dt.month/12))
    df['day_of_week'] = df['timestamp'].dt.dayofweek
    holidays_MA = holidays.US(years=range(2005, 2012), state="MA")
    df['is_holiday'] = df['timestamp'].dt.date.isin(holidays_MA.keys())
    df['cos_hour'] = np.cos(2*np.pi*(df['timestamp'].dt.hour/24))
    df['sin_hour'] = np.sin(2*np.pi*(df['timestamp'].dt.hour/24))
    #df['holiday_name'] = df.apply(lambda row: "None" if not df['is_holiday'] else holidays_MA[row["timestamp"].date()], axis=1): too slow, to be optimized
    df = df.drop(columns=["timestamp"])

    onehots = []
    if encode_meter_id:
      meter_onehot = pd.get_dummies(df['meter_id'], prefix="meter")
      onehots.append(meter_onehot)
    df = df.drop(columns=["meter_id", "mid_level", "aggregate"])
    dow_onehot = pd.get_dummies(df['day_of_week'], prefix="dow")
    df = df.drop(columns=["day_of_week"])
    features_df = pd.concat([df, dow_onehot] + onehots, axis=1)
    return features_df

In [8]:
top_level_df = feature_engineering(top_level_df, False)
for agg in aggregate_levels.keys():
  aggregate_levels[agg] = feature_engineering(aggregate_levels[agg], False)
for mid_level in mid_levels.keys():
  mid_levels[mid_level] = feature_engineering(mid_levels[mid_level], False)
for bottom_level in bottom_levels.keys():
  bottom_levels[bottom_level] = feature_engineering(bottom_levels[bottom_level], False)

In [9]:
def train_test_split(df):
    train_df = df[df["year"] < 2011].fillna(0)
    test_df = df[df["year"] == 2011].dropna()
    return train_df, test_df
top_level_train, top_level_test = train_test_split(top_level_df)
aggregate_train = {}
aggregate_test = {}
mid_levels_train = {}
mid_levels_test = {}
bottom_levels_train = {}
bottom_levels_test = {}
for agg in aggregate_levels.keys():
  train_df, test_df = train_test_split(aggregate_levels[agg])
  aggregate_train[agg] = train_df
  aggregate_test[agg] = test_df
for mid_level in mid_levels.keys():
  train_df, test_df = train_test_split(mid_levels[mid_level])
  mid_levels_train[mid_level] = train_df
  mid_levels_test[mid_level] = test_df
for bottom_level in bottom_levels.keys():
  train_df, test_df = train_test_split(bottom_levels[bottom_level])
  bottom_levels_train[bottom_level] = train_df
  bottom_levels_test[bottom_level] = test_df

# SARIMA with covariates

In [12]:
top_level_train.to_csv("data/temp/top_level_train.csv", index=False)
top_level_test.to_csv("data/temp/top_level_test.csv", index=False)

In [11]:
p, q, P, Q = (1,1,1,1)
endog = top_level_train["load"]
exog = top_level_train.drop(columns=["load", "cos_hour", "sin_hour", "dow_0"]).astype(float)
top_model = SARIMAX(endog, 
                            order=(p, 1, q),
                            seasonal_order=(P, 1, Q, 24),
                            freq = None,
                            exog = exog)
top_model = top_model.fit()
top_model.save('models/SARIMAX/top_model.pkl')

KeyboardInterrupt: 

In [ ]:
for agg in aggregate_train.keys():
  endog = aggregate_train[agg]["load"]
  exog = aggregate_train[agg].drop(columns="load").astype(float)
  agg_model = SARIMAX(endog,
                            order=(p, 1, q),
                            seasonal_order=(P, 1, Q, 24),
                            freq = None,
                            exog = exog)
  agg_model = agg_model.fit(method="powell")
  agg_model.save('models/SARIMAX/agg_model_'+ agg +'.pkl')

In [ ]:
for mid in mid_levels_train.keys():
  endog = mid_levels_train[mid]["load"]
  exog = mid_levels_train[mid].drop(columns="load").astype(float)
  mid_model = SARIMAX(endog,
                            order=(p, 1, q),
                            seasonal_order=(P, 1, Q, 24),
                            freq = None,
                            exog = exog)
  mid_model = mid_model.fit(method="powell")
  mid_model.save('models/SARIMAX/mid_model_'+ mid +'.pkl')

In [ ]:
for bottom in bottom_levels_train.keys():
  endog = bottom_levels_train[bottom]["load"]
  exog = bottom_levels_train[bottom].drop(columns="load").astype(float)
  bottom_model = SARIMAX(endog,
                            order=(p, 1, q),
                            seasonal_order=(P, 1, Q, 24),
                            freq = None,
                            exog = exog)
  bottom_model = bottom_model.fit(method="powell")
  bottom_model.save('models/SARIMAX/bottom_model_'+ bottom +'.pkl')

In [ ]:
# Encode the hierarchy information
meter_ids = pd.unique(joined_data_df["meter_id"])
mid_level_ids = pd.unique(joined_data_df["mid_level"])
aggregate_level_ids = pd.unique(joined_data_df["aggregate"])
n_series = 1 + len(aggregate_level_ids) + len(mid_level_ids) + len(meter_ids)
S = np.zeros((n_series, len(meter_ids)))
hierarchy_df = pd.read_csv("data/hierarchy.csv")
hierarchy_df = hierarchy_df[hierarchy_df["meter_id"].isin(meter_ids)]
hierarchy_df["indicator"] = 1
aggregate_indicators = hierarchy_df.pivot_table(index="aggregate", columns="meter_id", values="indicator", fill_value=0)
mid_level_indicators = hierarchy_df.pivot_table(index="mid_level", columns="meter_id", values="indicator", fill_value=0)

# rows / columns
row_names = ['Total'] + list(aggregate_level_ids) + list(mid_level_ids) + list(meter_ids)
S = pd.DataFrame(S, index=row_names, columns=meter_ids)

S.loc['Total', :] = 1
for agg in aggregate_level_ids:
  S.loc[agg] = aggregate_indicators.loc[agg]
for mid in mid_level_ids:
  S.loc[mid] = mid_level_indicators.loc[mid]
for meter_id in meter_ids:
  S.loc[meter_id, meter_id] = 1


tags = {}
tags['Total'] = np.array(['Total'], dtype=object)
tags['Total/Aggregate'] = aggregate_level_ids
tags['Total/Aggregate/MidLevel'] = mid_level_ids
tags['Total/Aggregate/MidLevel/Meter'] = meter_ids

In [ ]:
h = len(top_level_test.index)
T = len(top_level_train.index)
y_hat = np.zeros((n_series, h))
Y_test = np.zeros((n_series, h))
Y_train = np.zeros((n_series, T))
Y_hat_train = np.zeros((n_series, T))
i_series = 0
mid_level_dict = hierarchy_df[["meter_id", "mid_level"]].set_index("meter_id").to_dict()["mid_level"]

top_model = sm.load("models/SARIMAX/top_model.pkl")
y_test = top_level_test["load"]
X_test = top_level_test.drop(columns="load")
y_pred = top_model.predict(X_test)
y_train = top_level_train["load"]
X_train = top_level_train.drop(columns="load")
y_hat_train = top_model.predict(X_train)
assert(len(y_train) == T)
assert(len(y_pred) == h)
y_hat[i_series, :] = y_pred.copy()
Y_test[i_series, :] = y_test.to_numpy()
Y_train[i_series, :] = y_train.to_numpy()
Y_hat_train[i_series, :] = y_hat_train.copy()
i_series += 1

# Aggregate level
for agg in aggregate_level_ids:
  agg_model = sm.load('models/SARIMAX/agg_model_'+ agg +'.pkl')
  y_test = aggregate_test[agg]["load"]
  X_test = aggregate_test[agg].drop(columns="load")
  y_pred = agg_model.predict(X_test)
  y_train = aggregate_train[agg]["load"]
  X_train = aggregate_train[agg].drop(columns="load")
  y_hat_train = top_model.predict(X_train)
  assert(len(y_train) == T)
  assert(len(y_pred) == h)
  y_hat[i_series, :] = y_pred.copy()
  Y_test[i_series, :] = y_test.to_numpy()
  Y_train[i_series, :] = y_train.to_numpy()
  Y_hat_train[i_series, :] = y_hat_train.copy()
  i_series += 1

# Mid level
for mid in mid_level_ids:
  mid_model = sm.load('models/SARIMAX/mid_model_'+ mid +'.pkl')
  y_test = mid_levels_test[mid]["load"]
  X_test = mid_levels_test[mid].drop(columns="load")
  y_pred = mid_model.predict(X_test)
  y_train = mid_levels_train[mid]["load"]
  X_train = mid_levels_train[mid].drop(columns="load")
  y_hat_train = mid_model.predict(X_train)
  assert(len(y_train) == T)
  assert(len(y_pred) == h)
  y_hat[i_series, :] = y_pred.copy()
  Y_test[i_series, :] = y_test.to_numpy()
  Y_train[i_series, :] = y_train.to_numpy()
  Y_hat_train[i_series, :] = y_hat_train.copy()
  i_series += 1

# Bottom level
for bottom in meter_ids:
  bottom_model = sm.load('models/SARIMAX/bottom_model_'+ bottom +'.pkl')
  y_test = bottom_levels_test[bottom]["load"]
  X_test = bottom_levels_test[bottom].drop(columns="load")
  y_pred = bottom_model.predict(X_test)
  y_train = bottom_levels_train[bottom]["load"]
  X_train = bottom_levels_train[bottom].drop(columns="load")
  y_hat_train = bottom_model.predict(X_train)
  assert(len(y_train) == T)
  assert(len(y_pred) == h)
  y_hat[i_series, :] = y_pred.copy()
  Y_test[i_series, :] = y_test.to_numpy()
  Y_train[i_series, :] = y_train.to_numpy()
  Y_hat_train[i_series, :] = y_hat_train.copy()
  i_series += 1

In [ ]:
y_hat_df = pd.DataFrame(y_hat, index=row_names, columns=[dt.datetime(2011,1,1)+i*dt.timedelta(seconds=3600) for i in range(h)])
y_hat_df = pd.melt(y_hat_df.reset_index().rename(columns={"index": "unique_id"}), id_vars="unique_id", var_name="ds", value_name="boost").set_index("unique_id")
Y_test = pd.DataFrame(Y_test, index=row_names, columns=[dt.datetime(2011,1,1)+i*dt.timedelta(seconds=3600) for i in range(h)])
Y_test = pd.melt(Y_test.reset_index().rename(columns={"index": "unique_id"}), id_vars="unique_id", var_name="ds", value_name="y").set_index("unique_id")
Y_train = pd.DataFrame(Y_train, index=row_names, columns=[dt.datetime(2005,1,1)+i*dt.timedelta(seconds=3600) for i in range(T)])
Y_train = pd.melt(Y_train.reset_index().rename(columns={"index": "unique_id"}), id_vars="unique_id", var_name="ds", value_name="y").set_index("unique_id")
Y_hat_train = pd.DataFrame(Y_hat_train, index=row_names, columns=[dt.datetime(2005,1,1)+i*dt.timedelta(seconds=3600) for i in range(T)])
Y_hat_train = pd.melt(Y_hat_train.reset_index().rename(columns={"index": "unique_id"}), id_vars="unique_id", var_name="ds", value_name="boost").set_index("unique_id")
Y_train = pd.concat([Y_train, Y_hat_train["boost"]], axis=1)

In [ ]:
reconcilers = [MinTrace(method="ols"), MinTrace(method="wls_struct"), MinTrace(method="wls_var"), MinTrace(method="mint_shrink"), MinTrace(method="mint_cov"),  BottomUp(), TopDown(method="forecast_proportions")]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
y_hat_rec = hrec.reconcile(y_hat_df, S, tags, Y_train)

In [ ]:
def rmse(x,y):
    return np.sqrt(mean_squared_error(x,y))
evaluator = HierarchicalEvaluation(evaluators=[rmse, mean_absolute_error])
evaluator.evaluate(Y_hat_df = y_hat_rec, Y_test_df = Y_test,
                   tags=tags)

In [ ]:
y_hat = np.zeros((n_series, h))
Y_hat_train = np.zeros((n_series, T))
i_series = 0

with open('models/prophat/top_model.pkl', 'rb') as model_file:
    top_model = pickle.load(model_file)
test_df = top_level_test.rename(columns={'timestamp': 'ds', 'load': 'y'})
y_pred = top_model.predict(test_df)
train_df = top_level_train.rename(columns={'timestamp': 'ds', 'load': 'y'})
y_hat_train = top_model.predict(train_df)
y_hat[i_series, :] = y_pred.copy()
Y_hat_train[i_series, :] = y_hat_train.copy()
i_series += 1

# Aggregate level
for agg in aggregate_level_ids:
  with open("models/prophet/agg_model_"+agg+".pkl", "rb") as model_file:
    agg_model = pickle.load(model_file)
  test_df = aggregate_test[agg].rename(columns={'timestamp': 'ds', 'load': 'y'})
  y_pred = agg_model.predict(test_df)
  train_df = aggregate_train[agg].rename(columns={"timestamp": "ds", "load": "y"})
  y_hat_train = agg_model.predict(X_train)
  y_hat[i_series, :] = y_pred.copy()
  Y_hat_train[i_series, :] = y_hat_train.copy()
  i_series += 1

# Mid level
for mid in mid_level_ids:
  with open("models/prophet/mid_model_"+mid+".pkl", "rb") as model_file:
    mid_model = pickle.load(model_file)
  test_df = mid_levels_test[mid].rename(columns={'timestamp': 'ds', 'load': 'y'})
  y_pred = mid_model.predict(test_df)
  train_df = mid_levels_train[mid].rename(columns={"timestamp": "ds", "load": "y"})
  y_hat_train = mid_model.predict(X_train)
  y_hat[i_series, :] = y_pred.copy()
  Y_hat_train[i_series, :] = y_hat_train.copy()
  i_series += 1

# Bottom level
for bottom in meter_ids:
  with open("models/prophet/bottom_model_"+bottom+".pkl", "rb") as model_file:
    bottom_model = pickle.load(model_file)
  test_df = bottom_levels_test[bottom].rename(columns={'timestamp': 'ds', 'load': 'y'})
  y_pred = bottom_model.predict(test_df)
  train_df = bottom_levels_train[bottom].rename(columns={"timestamp": "ds", "load": "y"})
  y_hat_train = bottom_model.predict(X_train)
  y_hat[i_series, :] = y_pred.copy()
  Y_hat_train[i_series, :] = y_hat_train.copy()
  i_series += 1

In [ ]:
reconcilers = [MinTrace(method="ols"), MinTrace(method="wls_struct"), MinTrace(method="wls_var"), MinTrace(method="mint_shrink"), MinTrace(method="mint_cov"),  BottomUp(), TopDown(method="forecast_proportions")]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)
y_hat_rec = hrec.reconcile(y_hat_df, S, tags, Y_train)

In [ ]:
evaluator = HierarchicalEvaluation(evaluators=[rmse, mean_absolute_error])
evaluator.evaluate(Y_hat_df = y_hat_rec, Y_test_df = Y_test,
                   tags=tags)